### Installation

In [ ]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth
!pip install --upgrade "transformers>=4.53.0"

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-09 18:52:35.930799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752087156.116909      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752087156.167789      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.1: Fast Mistral patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.7.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from transformers.models.mistral.modeling_mistral import MistralAttention
model_name = "mistral"
rope_module = None
scaled_rope_module = None
attention_module = MistralAttention

rope_name = "LlamaRotaryEmbedding"
scaled_rope_name = "LlamaLinearScalingRotaryEmbedding"
model_filepath = f"transformers.models.{model_name}.modeling_{model_name}"
exec_code = \
    f"import torch.nn as nn\n"\
    f"from typing import Union, Optional, List, Any, Callable, Tuple\n"\
    f"from {model_filepath} import logger, "\
    f"{model_name.title()}Attention, {model_name.title()}Config"
exec(exec_code)

### Data Preparation

#### Load MetaMath

In [ ]:
from datasets import load_dataset
dataset = load_dataset("meta-math/MetaMathQA", split="train[:100000]")

README.md: 0.00B [00:00, ?B/s]

MetaMathQA-395K.json:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/395000 [00:00<?, ? examples/s]

#### Split Train Val

In [ ]:
split_data = dataset.train_test_split(test_size=0.3, seed=42)

Chat Template Format
```python
[
    {"role": "system", "content": """Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers."""},
    {"role": "user", "content": "{original_question}"},
    {"role": "assistant", "content": "{response}"}
]
```

#### Prep Data

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
    map_eos_token = True,
)

# System Instruction cố định cho tất cả các cuộc hội thoại
system_instruction_fixed = """Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers."""

def formatting_prompts_func_conversational_structured(examples):
    queries            = examples["query"]
    responses          = examples["response"]

    texts = []

    for i in range(len(queries)):
        # Tạo danh sách các dictionary messages cho mỗi cuộc hội thoại
        conversation_messages = []

        # Thêm System Message
        conversation_messages.append({"role": "system", "content": system_instruction_fixed})

        # Thêm User Message (từ 'query')
        user_content = queries[i]
        conversation_messages.append({"role": "user", "content": user_content})

        # Thêm Assistant Message (từ 'response')
        assistant_content = responses[i]
        conversation_messages.append({"role": "assistant", "content": assistant_content})

        # Áp dụng chat_template để biến danh sách messages thành một chuỗi text duy nhất
        tokenized_text = tokenizer.apply_chat_template(
            conversation_messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(tokenized_text)

    return { "text" : texts }

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
dataset_dict = split_data.map(formatting_prompts_func_conversational_structured, batched=True)
train_dataset = dataset_dict["train"]
val_dataset = dataset_dict["test"]

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[5]["text"])

<|im_start|>system
Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers.<|im_end|>
<|im_start|>user
What is the value of $a$ such that the graph of the equation $x^2 + 4y^2 - 10x + 56y = k$ represents a non-degenerate ellipse only when $k > a$?<|im_end|>
<|im_start|>assistant
To determine the shape of the graph, we can complete the square for both the $x$ and $y$ terms.
$x^2 - 10x$ can be rewritten as $(x-5)^2 - 25$ by adding and subtracting 25.
$4y^2 + 56y$ can be rewritten as $4(y+7)^2 - 196$ by adding and subtracting 196/4.
Substituting these expressions back into the equation, we have $(x-5)^2 - 25

### Wandb

In [ ]:
import wandb

wandb.login(key="wandb-key")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thnndat236 (thnndat236-tr-ng-i-h-c-khoa-h-c-t-nhi-n-hqg-hcm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os

os.environ["WANDB_PROJECT"] = "my-finetune-metamath-100k"
os.environ["WANDB_ENTITY"] = "thnndat236-tr-ng-i-h-c-khoa-h-c-t-nhi-n-hqg-hcm"

### Train the model

In [ ]:
from trl import SFTConfig, SFTTrainer
# from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1300,
        learning_rate = 2e-4,
        logging_steps = 1,
        # eval_strategy="steps",
        # eval_steps = 2,
        # per_device_eval_batch_size = 2,
        # save_strategy = "steps",
        # save_steps = 2,
        # load_best_model_at_end=True,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/70000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/30000 [00:00<?, ? examples/s]

#### Show current memory stats

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.883 GB of memory reserved.


#### Train

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 70,000 | Num Epochs = 1 | Total steps = 1,300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080 of 7,331,909,632 (1.14% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250709_185428-7vbbow6y
wandb: Run `wandb offli

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.722900
2,1.795800
3,1.290300
4,1.156700
5,0.950500
6,0.731900
7,0.541000
8,0.527500
9,0.467300
10,0.450000


#### Show final memory and time stats

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

17490.4127 seconds used for training.
291.51 minutes used for training.
Peak reserved memory = 6.883 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 46.693 %.
Peak reserved memory for training % of max memory = 0.0 %.


### Inference

In [ ]:
test_question = "If Josh spent $1.75 on a drink and then spent an additional $1.25, and he has $6 left, what was the initial amount of money that Josh had, in dollars?"

#### Inference Model

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "system", "content": system_instruction_fixed},
    {"role": "user", "content": test_question},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers.<|im_end|>
<|im_start|>user
If Josh spent $1.75 on a drink and then spent an additional $1.25, and he has $6 left, what was the initial amount of money that Josh had, in dollars?<|im_end|>
<|im_start|>assistant
Josh spent $1.75 on a drink and then spent an additional $1.25, so he spent a total of $1.75 + $1.25 = $3.
If he has $6 left, then the initial amount of money that Josh had is $6 + $3 = $9.
#### 9
The answer is: 9<|im_end|>


### Save model on HF

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [ ]:
model.push_to_hub("2phonebabykeem/mistral_math_finetune_wandb", token="hf_token")
tokenizer.push_to_hub("2phonebabykeem/mistral_math_finetune_wandb", token="hf_token")

README.md:   0%|          | 0.00/613 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Saved model to https://huggingface.co/2phonebabykeem/mistral_math_finetune_wandb


No files have been modified since last commit. Skipping to prevent empty commit.
